In [ ]:
using ITensors, ITensorMPS, PolyChaos, Plots, Base.Threads
Threads.nthreads()

Variables

In [ ]:
beta = 10 #inverse temperature 
mu = 0.0 #chemical potential
N = 50 #number of sites in each chain
Es = 0.0 #system site energy
D = 1. #bandwidth
J_0 = 0.005 #system-bath coupling prefactor

Spectral function, thermofield transformation of spectral function, orthogonal polynomial chain mapping

In [ ]:
J(omega) = (-1. <= omega < 1.) ? J_0 * sqrt(1 - (omega/D)^2) : 0.0  #g(x) = x so h^2(x) = J(x)                                             

J1(omega) = J(omega) * fermi(omega) #effective empty mode spectral density
J2(omega) = J(omega) * (1 - fermi(omega)) #effective filled mode spectral density

fermi(k) = 1/(1 + exp(beta*k - beta*mu)) #fermi-dirac distribution

function chain_map(J, supp=(-1,1))
    """calculates family of monic orthogonal polynomials w.r.t the measure J(x) up to the Nth term.
    returns the coefficients alpha and beta from the recurrence relation of the family."""
    meas = Measure("bath", J, supp, false, Dict())
    ortho_poly = OrthoPoly("bath_op", N, meas; Nquad=2000)   
    AB = coeffs(ortho_poly)                                  
    return AB                             
end

generation and plotting of chain coefficients (site energies and hoppings)

In [ ]:
chain1 = chain_map(J1) #coefficients from empty chain polynomials
chain2 = chain_map(J2) #coefficients from filled chain polynomials

energies1 = chain1[:,1] #on-site energies of empty chain
hopping1 = sqrt.(chain1[2:N+1,2]) #nearest neighbour hoppings of empty chain

energies2 = chain2[:,1] #on-site energies of filled chain
hopping2 = sqrt.(chain2[2:N+1,2]) #nearest neighbour hoppings of filled chain

p1 = plot(energies1, xlabel="n", ylabel="energy", label="filled chain")
plot!(p1, energies2, label="empty chain")
p2 = plot(hopping1, xlabel="n", ylabel="hopping", label="filled chain")
plot!(p2, hopping2, label="empty chain")

display(plot(p1,p2))


creation of initial state MPS and Hamiltonian MPO

In [ ]:
sites = siteinds("Fermion", 2*N+1, conserve_qns=true) #assuming that truncated bath remains approximately closed within simulation time

states = [(j <= N) ? "Occ" : "Emp" for j=1:2*N+1]

psi0 = productMPS(sites, states)

init_occ = expect(psi0, "N")
display(plot(1:(2N+1), init_occ),xlabel="site",ylabel="occupation",title="initial occupations")

sys = N+1

function Hamiltonian(E1, E2, h1, h2, sites, Es, N, sys)
    ampo = AutoMPO()
    t1 = reverse(h1[2:end]) #empty chain NN couplings
    t2 = reverse(h2[2:end]) #filled chain NN couplings
    g1 = h1[1] #empty chain-system coupling
    g2 = h2[1] #filled chain-system coupling
    
    println("system-bath couplings are $g1 and $g2")
    for j in 1:N
        add!(ampo, E1[j], "N", j)
    end
    for j in 1:N
        add!(ampo, E1[j], "N", j)
    end
    for j in 1:N-1
        add!(ampo, t1[j], "Cdag", j, "C", j+1)
        add!(ampo, t1[j], "Cdag", j+1, "C", j)
    end
    for j in 1:N
        add!(ampo, E2[j], "N", sys + j)
    end
    for j in 1:N-1
        add!(ampo, t2[j], "Cdag", sys + j, "C", sys + j + 1)
        add!(ampo, t2[j], "Cdag", sys + j + 1, "C", sys + j)
    end
    # system onsite
    add!(ampo, Es, "N", sys)
    add!(ampo, g1, "Cdag", sys, "C", N);         add!(ampo, g1, "Cdag", N, "C", sys)
    add!(ampo, g2, "Cdag", sys, "C", sys + 1);    add!(ampo, g2, "Cdag", sys + 1,  "C", sys)
    return MPO(ampo, sites)
end

TDVP time evolution of MPS via Hamiltonian MPO

In [ ]:
sweeps = Sweeps(2); maxdim!(sweeps, 400, 800); cutoff!(sweeps, 1e-9)
H = Hamiltonian(energies1, energies2, hopping1, hopping2, sites, 0.0, N, sys)
println(H)
let
    psi = psi0
    dt = 0.1
    tmax = 7
    ts = collect(dt:dt:tmax)
    len = length(ts)
    nSys = zeros(len)
    nCheck = zeros(len)
    for k in 1:len
        psi = ITensorMPS.tdvp(H, -im*dt, psi;nsite=2, outputlevel=0,mindim=1, maxdim=100); #time_step=dt, nsweeps=sweeps, order=2)
        num = expect(psi, "N")
        nSys[k] = num[sys]
        nCheck[k] = num[2N+1]
        println("timstep $k of $len complete")

    end
    display(plot(ts,nSys,xlabel="time",ylabel="occupation",title=r"$N=$N, \\beta=$beta, \\mu=$mu$"))
    display(plot(ts,nCheck,xlabel="time",ylabel="occupation",title="check occupation of last site"))    
end 